In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.width", None)  # Set width to expand to full display
pd.set_option("display.max_colwidth", None)

In [ ]:
# Load the user's uploaded file
file_path = "./Actual Generation per Production Type_202401010000-202401020000.csv"  # this is portugal data
data = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
data.head()

In [ ]:
# Filter columns that represent "Actual Aggregated [MW]" values
aggregated_columns = [col for col in data.columns if "Actual Aggregated [MW]" in col]

# Convert columns to numeric, replacing non-numeric values (like 'n/e') with 0, then sum per row
data[aggregated_columns] = (
    data[aggregated_columns].apply(pd.to_numeric, errors="coerce").fillna(0)
)
data["Total Actual Aggregated [MW]"] = data[aggregated_columns].sum(axis=1)

data["start_time"] = pd.to_datetime(
    data["MTU"].str.split(" - ").str[0], format="%d.%m.%Y %H:%M"
)
data.set_index("start_time", inplace=True)

# Display the resulting sum for each time interval
data[["MTU", "Total Actual Aggregated [MW]"]].head()

In [ ]:
file_path_2 = "./Actual Generation per Production Type_202401010000-202401020000 (1).csv"  # this is spain data
data_2 = pd.read_csv(file_path_2)
data_2.head()

In [ ]:
numbers = [
    float(x)
    for x in "235.0	0.0	0.0	4331.0	275.0	20.0	0.0	0.0	0.0	663.5	964.0	2011.0	0.0	7096.0	74.0	28.0	247.0	0.0	5656.0	0.0".split()
]
total = sum(numbers)
print(f"Sum: {total}")

In [ ]:
# Parse the 'MTU' column to extract date and time for resampling
# Extract start time of each interval from 'MTU' and convert to datetime
data_2["Start Time"] = pd.to_datetime(
    data_2["MTU"].str.split(" - ").str[0], format="%d.%m.%Y %H:%M"
)

# Set 'Start Time' as index for resampling
data_2.set_index("Start Time", inplace=True)

# Filter columns that represent "Actual Aggregated [MW]" values
aggregated_columns_2 = [
    col for col in data_2.columns if "Actual Aggregated [MW]" in col
]

# Convert columns to numeric, handling non-numeric values
data_2[aggregated_columns_2] = (
    data_2[aggregated_columns_2].apply(pd.to_numeric, errors="coerce").fillna(0)
)

# Resample to hourly intervals by taking the mean within each hour
data_2_hourly = data_2[aggregated_columns_2].resample("H").mean()

data_2_hourly.head()

# # Calculate total "Actual Aggregated [MW]" by summing across all sources for each hour
# data_hourly['Total Actual Aggregated [MW]'] = data_hourly.sum(axis=1)

# # Display the resulting sum for each hour
# data_hourly[['Total Actual Aggregated [MW]']].head()

In [ ]:
data_2_hourly.sum(axis=1)

In [ ]:
# Calculate the nuclear fraction for each hour
nuclear_column = [col for col in data_2_hourly.columns if "Nuclear" in col][
    0
]  # Get the nuclear column name
data_2_hourly["Total Generation"] = data_2_hourly.sum(
    axis=1
)  # Calculate total generation
data_2_hourly["Nuclear Fraction"] = (
    data_2_hourly[nuclear_column] / data_2_hourly["Total Generation"]
)

# Display the nuclear fraction for each hour
print("Nuclear fraction of total generation by hour:")
print(data_2_hourly["Nuclear Fraction"].head())

In [ ]:
# data_2_hourly[nuclear_column]
data_2_hourly.iloc[0]

In [ ]:
import pandas as pd

# Load the user's uploaded file
file_path = "./Cross-Border Physical Flow_202401010000-202401020000.csv"
data_3 = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
data_3.head()

In [ ]:
# Parse the time column to get start times
data_3["start_time"] = pd.to_datetime(
    data_3["Time (UTC)"].str.split(" - ").str[0], format="%d.%m.%Y %H:%M"
)

# Set start_time as index to align with nuclear fraction data_3
data_3.set_index("start_time", inplace=True)

# Calculate nuclear power flow from ES to PT
# Nuclear fraction * Flow from ES to PT
nuclear_flow = data_3["BZN|ES > BZN|PT [MW]"] * data_2_hourly["Nuclear Fraction"]

print("\nEstimated nuclear power flow from ES to PT (MW):")
print(nuclear_flow.head())

In [ ]:
# Calculate total Portuguese power consumption
# Total consumption = Power generated in Portugal + Power flowing from Spain - Power flowing to Spain
portuguese_consumption = (
    data["Total Actual Aggregated [MW]"]
    + data_3["BZN|ES > BZN|PT [MW]"]
    - data_3["BZN|PT > BZN|ES [MW]"]
)

# Calculate percentage of nuclear power in Portuguese consumption
nuclear_percentage = (nuclear_flow / portuguese_consumption) * 100

print("\nPercentage of Portuguese power consumption from Spanish nuclear:")
print(nuclear_percentage.head())
print("\nSummary statistics:")
print(f"Mean: {nuclear_percentage.mean():.2f}%")
print(f"Max: {nuclear_percentage.max():.2f}%")
print(f"Min: {nuclear_percentage.min():.2f}%")